In [1]:
import sys
sys.path.append(r'C:\Users\veena\Downloads\bomberman_rl-master')


In [2]:
from environment import BombeRLeWorld, WorldArgs
import events as e 



In [3]:
from environment import WorldArgs  

In [4]:
args = WorldArgs(
    no_gui=True,                # Set True to run without GUI
    fps=30,                     # Frames per second
    turn_based=False,            # Whether the game is turn-based
    update_interval=0.1,         # Interval between updates in seconds
    save_replay=False,           # Whether to save a replay of the game
    replay=None,                 # Replay file (if any)
    make_video=False,            # Whether to record the game as a video
    continue_without_training=False, # Continue the game without training
    log_dir="C:/Users/veena/Downloads/bomberman_rl-master/bomberman_rl-master/logs",  # Set the log directory
    save_stats=False,            # Whether to save game statistics
    match_name="Test Match",     # Name of the match
    seed=42,                     # Random seed for reproducibility
    silence_errors=False,        # Whether to silence errors
    scenario="treasure-hunt"              # Scenario for the game
)

# Create a list of agents (could be AI agents or rule-based)
agents = []  # Add your agents here

# Initialize the environment with the specified arguments and agents
env = BombeRLeWorld(args, agents)


AttributeError: 'BombeRLeWorld' object has no attribute 'rng'

In [ ]:
log_dir="C:/Users/veena/Downloads/bomberman_rl-master/bomberman_rl-master/logs",

In [ ]:
import numpy as np

class QLearningAgent:
    def __init__(self, learning_rate=0.1, discount_factor=0.9, exploration_rate=0.1):
        self.q_table = {}  # This will store state-action pairs and their Q-values
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate

    def state_to_tuple(self, state):
        """Convert the game state dictionary into a hashable tuple, including handling numpy arrays and lists."""
        def convert_value(value):
            # Convert numpy arrays to tuples
            if isinstance(value, np.ndarray):
                return tuple(value.flatten())  # Flatten to 1D and convert to tuple
            # Convert lists to tuples
            elif isinstance(value, list):
                return tuple(convert_value(v) for v in value)  # Recursively convert list elements
            # Convert nested dictionaries
            elif isinstance(value, dict):
                return self.state_to_tuple(value)  # Recursively convert dictionaries
            return value

        # Convert all state values to hashable types (like tuples)
        return tuple((k, convert_value(v)) for k, v in state.items())

    def predict(self, state):
        """Choose the best action based on current Q-values"""
        # Convert the state to a hashable type
        state_tuple = self.state_to_tuple(state)

        if state_tuple not in self.q_table:
            self.q_table[state_tuple] = {action: 0 for action in ['UP', 'DOWN', 'LEFT', 'RIGHT', 'BOMB', 'WAIT']}

        # Epsilon-greedy action selection (explore vs exploit)
        if np.random.rand() < self.exploration_rate:
            # Explore: Take a random action
            return np.random.choice(['UP', 'DOWN', 'LEFT', 'RIGHT', 'BOMB', 'WAIT'])
        else:
            # Exploit: Take the action with the highest Q-value
            return max(self.q_table[state_tuple], key=self.q_table[state_tuple].get)

    def update(self, state, action, reward, next_state):
        """Update the Q-value based on the received reward"""
        # Convert the states to hashable types
        state_tuple = self.state_to_tuple(state)
        next_state_tuple = self.state_to_tuple(next_state)

        if state_tuple not in self.q_table:
            self.q_table[state_tuple] = {a: 0 for a in ['UP', 'DOWN', 'LEFT', 'RIGHT', 'BOMB', 'WAIT']}
        if next_state_tuple not in self.q_table:
            self.q_table[next_state_tuple] = {a: 0 for a in ['UP', 'DOWN', 'LEFT', 'RIGHT', 'BOMB', 'WAIT']}

        # Q-Learning update rule
        best_next_action = max(self.q_table[next_state_tuple], key=self.q_table[next_state_tuple].get)
        current_q = self.q_table[state_tuple][action]
        max_future_q = self.q_table[next_state_tuple][best_next_action]

        # Update Q-value
        new_q = current_q + self.learning_rate * (reward + self.discount_factor * max_future_q - current_q)
        self.q_table[state_tuple][action] = new_q



In [ ]:
from tqdm import tqdm

In [ ]:
from tqdm import tqdm  # Import tqdm correctly

def train_agent(agent, env, num_episodes=1000):
    for episode in tqdm(range(num_episodes)):
        game_state = env.reset()  # Reset the environment at the start of each episode
        done = False
        
        while not done:
            # The agent chooses an action based on the current game state
            action = agent.predict(game_state)
            
            # Perform the action in the environment
            new_game_state, reward, done, info = env.step(action)  # Using the new step method
            
            # Update the agent's Q-values based on the game state and reward
            agent.update(game_state, action, reward, new_game_state)
            
            # Move to the next state
            game_state = new_game_state


In [ ]:
# Initialize the Q-learning agent
agent = QLearningAgent()

# Train the agent in the environment
train_agent(agent, env, num_episodes=1000)



In [ ]:
import numpy as np
import pygame
import tqdm


In [ ]:
class QLearningAgent:
    def __init__(self, learning_rate=0.1, discount_factor=0.9, exploration_rate=0.1):
        self.q_table = {}  # Q-table for state-action pairs
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate

    def predict(self, state):
        """Choose the best action based on current Q-values"""
        if state not in self.q_table:
            self.q_table[state] = {action: 0 for action in ['UP', 'DOWN', 'LEFT', 'RIGHT', 'BOMB', 'WAIT']}
        
        # Epsilon-greedy action selection (explore vs exploit)
        if np.random.rand() < self.exploration_rate:
            # Explore: Take a random action
            return np.random.choice(['UP', 'DOWN', 'LEFT', 'RIGHT', 'BOMB', 'WAIT'])
        else:
            # Exploit: Take the action with the highest Q-value
            return max(self.q_table[state], key=self.q_table[state].get)

    def update(self, state, action, reward, next_state):
        """Update the Q-value based on the received reward"""
        if state not in self.q_table:
            self.q_table[state] = {a: 0 for a in ['UP', 'DOWN', 'LEFT', 'RIGHT', 'BOMB', 'WAIT']}
        if next_state not in self.q_table:
            self.q_table[next_state] = {a: 0 for a in ['UP', 'DOWN', 'LEFT', 'RIGHT', 'BOMB', 'WAIT']}
        
        # Q-Learning update rule
        best_next_action = max(self.q_table[next_state], key=self.q_table[next_state].get)
        current_q = self.q_table[state][action]
        max_future_q = self.q_table[next_state][best_next_action]
        
        # Update Q-value
        new_q = current_q + self.learning_rate * (reward + self.discount_factor * max_future_q - current_q)
        self.q_table[state][action] = new_q


In [ ]:
def extract_features(game_state):
    """Extract features such as the agent's position, nearby coins, and bombs"""
    # The agent's current position (x, y)
    _, _, _, (x, y) = game_state['self']
    
    # List of coins on the board
    coins = game_state['coins']
    
    # Calculate distance to the closest coin
    coin_distances = [np.linalg.norm(np.array([x, y]) - np.array(coin)) for coin in coins]
    closest_coin_distance = min(coin_distances) if coin_distances else float('inf')
    
    return (x, y, closest_coin_distance)


In [ ]:
def act(agent, game_state):
    """Make the agent choose an action based on the game state"""
    features = extract_features(game_state)
    action = agent.predict(features)
    return action


In [ ]:
def train_agent(agent, env, num_episodes=1000):
    for episode in tqdm.tqdm(range(num_episodes)):
        game_state = env.reset()  # Start a new game
        done = False
        
        while not done:
            # The agent chooses an action based on the current game state
            action = act(agent, game_state)
            
            # Perform the action in the environment
            new_game_state, reward, done, info = env.step(action)
            
            # Update the Q-learning model based on the reward and the new state
            features = extract_features(game_state)
            new_features = extract_features(new_game_state)
            agent.update(features, action, reward, new_features)
            
            # Move to the next state
            game_state = new_game_state
